# Load a model

In [163]:
import spacy
nlp = spacy.load('en_core_web_sm')

# Some imports

In [164]:
from spacy import displacy
# from __future__ import unicode_literals, print_function
import random
from pathlib import Path
from tqdm import tqdm

# What does the model recognise out of the box?

In [165]:
doc = nlp("Who is David Doherty? Know if he trades interest rate swaps??")

In [166]:
for token in doc.ents:
    print(token.text, token.start_char, token.label_)

David Doherty 7 PERSON


We're famous!

In [167]:
displacy.render(doc, style='ent', jupyter=True)

# But DeepGreen is a product not a person....

In [189]:
sentence_label_pairs = [
    {'sentence':"Who is David Doherty? Know if he trades interest rate swaps??", 'labels': [{"label":"interest rate swaps", "entity":"FINANCIAL_PRODUCT"}]},
    {'sentence':"Some guy from JPM called about an interest rate swap or something??", 'labels': [{"label":"interest rate swap", "entity":"FINANCIAL_PRODUCT"}]},
    {'sentence':"Anyone called you about a credit default swap today? Seems quiet", 'labels': [{"label":"credit default swap", "entity":"FINANCIAL_PRODUCT"}]},
    {'sentence':"Anyone called you about a interest rate swap today? Seems quiet", 'labels': [{"label":"interest rate swap", "entity":"FINANCIAL_PRODUCT"}]},
    {'sentence':"Derivatives market are made up of many products like IRS, CDS, CDO, etc", 'labels': [{"label":"IRS", "entity":"FINANCIAL_PRODUCT"}, {"label":"CDS", "entity":"FINANCIAL_PRODUCT"}, {"label":"CDO", "entity":"FINANCIAL_PRODUCT"}]},
    {'sentence':"To hedge rates positions you can use IRS which trade with lots of liquidity", 'labels': [{"label":"IRS", "entity":"FINANCIAL_PRODUCT"}]},
    {'sentence':"Beyond the liquid futures market, most use IRS to position themselves for increasing yields", 'labels': [{"label":"IRS", "entity":"FINANCIAL_PRODUCT"}]},
    {'sentence':"Though the swap curve uses many IRS instruments, they are normally spread against liquid bond markets", 'labels': [{"label":"IRS", "entity":"FINANCIAL_PRODUCT"}]},
    {'sentence':"A butterfly trade is simply 3 IRS at different maturities", 'labels': [{"label":"IRS", "entity":"FINANCIAL_PRODUCT"}]},
    {'sentence':"Roses are red, violets are blue, IRS is for rates, CDS is for credit too", 'labels': [{"label":"IRS", "entity":"FINANCIAL_PRODUCT"}, {"label":"CDS", "entity":"FINANCIAL_PRODUCT"}]}
]

def get_labels_for_training(sentence_label_pairs):
    output = []
    for pair in sentence_label_pairs:
        entities = []
        print(pair['sentence'])
        for labels in pair['labels']:
            start_index = pair['sentence'].index(labels['label'])
            end_index = start_index + len(labels['label'])
            entities.append((start_index, end_index, labels['entity']))
        output.append((pair['sentence'], {'entities':entities}))
            
    return output
        
TRAIN_DATA = get_labels_for_training(sentence_label_pairs)

Who is David Doherty? Know if he trades interest rate swaps??
Some guy from JPM called about an interest rate swap or something??
Anyone called you about a credit default swap today? Seems quiet
Anyone called you about a interest rate swap today? Seems quiet
Derivatives market are made up of many products like IRS, CDS, CDO, etc
To hedge rates positions you can use IRS which trade with lots of liquidity
Beyond the liquid futures market, most use IRS to position themselves for increasing yields
Though the swap curve uses many IRS instruments, they are normally spread against liquid bond markets
A butterfly trade is simply 3 IRS at different maturities
Roses are red, violets are blue, IRS is for rates, CDS is for credit too


In [190]:
TRAIN_DATA[:5]

[('Who is David Doherty? Know if he trades interest rate swaps??',
  {'entities': [(40, 59, 'FINANCIAL_PRODUCT')]}),
 ('Some guy from JPM called about an interest rate swap or something??',
  {'entities': [(34, 52, 'FINANCIAL_PRODUCT')]}),
 ('Anyone called you about a credit default swap today? Seems quiet',
  {'entities': [(26, 45, 'FINANCIAL_PRODUCT')]}),
 ('Anyone called you about a interest rate swap today? Seems quiet',
  {'entities': [(26, 44, 'FINANCIAL_PRODUCT')]}),
 ('Derivatives market are made up of many products like IRS, CDS, CDO, etc',
  {'entities': [(53, 56, 'FINANCIAL_PRODUCT'),
    (58, 61, 'FINANCIAL_PRODUCT'),
    (63, 66, 'FINANCIAL_PRODUCT')]})]

In [191]:
ner = nlp.get_pipe('ner')

In [192]:
ner.cfg['extra_labels'] = ['AWESOME_PRODUCT', 'ASSET_CLASS', 'FINANCIAL_PRODUCT', 'MARKET_DATA']
ner.cfg

{'beam_density': 0.0,
 'beam_width': 1,
 'cnn_maxout_pieces': 3,
 'deprecation_fixes': {'vectors_name': None},
 'extra_labels': ['AWESOME_PRODUCT',
  'ASSET_CLASS',
  'FINANCIAL_PRODUCT',
  'MARKET_DATA'],
 'hidden_depth': 1,
 'hidden_width': 200,
 'hist_size': 0,
 'hist_width': 0,
 'maxout_pieces': 2,
 'nr_class': 73,
 'pretrained_dims': 0,
 'pretrained_vectors': None,
 'token_vector_width': 128}

In [194]:
n_iter = 20
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            print(text, annotations)
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
#         print(losses)

  0%|          | 0/10 [00:00<?, ?it/s]

Anyone called you about a credit default swap today? Seems quiet {'entities': [(26, 45, 'FINANCIAL_PRODUCT')]}


 10%|█         | 1/10 [00:00<00:03,  2.65it/s]

A butterfly trade is simply 3 IRS at different maturities {'entities': [(30, 33, 'FINANCIAL_PRODUCT')]}


 20%|██        | 2/10 [00:00<00:03,  2.33it/s]

...

 80%|████████  | 8/10 [00:02<00:00,  2.75it/s]

Derivatives market are made up of many products like IRS, CDS, CDO, etc {'entities': [(53, 56, 'FINANCIAL_PRODUCT'), (58, 61, 'FINANCIAL_PRODUCT'), (63, 66, 'FINANCIAL_PRODUCT')]}


 90%|█████████ | 9/10 [00:02<00:00,  3.13it/s]

Beyond the liquid futures market, most use IRS to position themselves for increasing yields {'entities': [(43, 46, 'FINANCIAL_PRODUCT')]}


100%|██████████| 10/10 [00:03<00:00,  3.25it/s]


Now let's check the results

In [203]:
doc = nlp("What on earth is an IRS interest rate swap!")

displacy.render(doc, style='ent', jupyter=True)

In [197]:
doc = nlp("Do you trade credit default swaps more than IRS?")

displacy.render(doc, style='ent', jupyter=True)